In [1]:
%matplotlib inline

In [5]:
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions

import numpy as np

from pilco.policies import RBFPolicy, TransformedPolicy

from pilco.transforms import SineTransform, CosineTransform

from pilco.agents.agents import EQGPAgent
from pilco.costs.costs import EQCost
from pilco.environments import Environment

from pilco.utils import get_complementary_indices

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from tqdm import trange

In [37]:
loc = tf.range(3) + 1
cov = loc[None, :] * loc[:, None]

rep = 3

indices = tf.convert_to_tensor([0], dtype=tf.int32)[:, None]


In [46]:
cov

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[1, 2, 3],
       [2, 4, 6],
       [3, 6, 9]], dtype=int32)>

In [ ]:
def recursive_replicate(loc, cov, indices):
    
    indices_a = indices
    indices_b = get_complementary_indices(indices[:, 0], 3)[:, None]

    indices_aa = tf.stack(tf.meshgrid(indices_a, indices_a), axis=2)
    indices_bb = tf.stack(tf.meshgrid(indices_b, indices_b), axis=2)

    indices_ab = tf.stack(tf.meshgrid(indices_b, indices_a), axis=2)
    indices_ba = tf.stack(tf.meshgrid(indices_a, indices_b), axis=2)
    
    loc_a = tf.gather_nd(loc, indices_a)
    loc_b = tf.gather_nd(loc, indices_b)
    
    rep_loc_a = tf.tile(loc_a, [rep])
    
    rep_loc = tf.concat([rep_loc_a])
    
    cov_aa = tf.gather_nd(cov, indices_aa) 
    cov_ab = tf.gather_nd(cov, indices_ab) 
    cov_ba = tf.gather_nd(cov, indices_ba) 
    cov_bb = tf.gather_nd(cov, indices_bb) 
    
    rep_cov_aa = tf.tile(cov_aa, [rep, rep])
    rep_cov_ab = tf.tile(cov_ab, [rep, 1])
    rep_cov_ba = tf.tile(cov_ba, [1, rep])

    row_blocks = [
        tf.concat([rep_cov_aa, rep_cov_ab], axis=1),
        tf.concat([rep_cov_ba, cov_bb], axis=1),
    ]

    rep_cov = tf.concat(row_blocks, axis=0)
    rep_cov

# Policy: match moments (closed form and MC)

## RBF Policy

In [3]:
rbf_policy = RBFPolicy(2, 1, 5, dtype=tf.float32)
rbf_policy.reset()

In [4]:
loc = tf.zeros(2, dtype=tf.float32)
cov = tf.eye(2, dtype=tf.float32)

mean_full, cov_full = rbf_policy.match_moments(loc, cov)

print('All eigenvalues are postive:', bool(tf.reduce_all(tf.cast(tf.linalg.eig(cov_full)[0], dtype=tf.float32) > 0)))

print(f'mean_full:\n{mean_full.numpy()}')
print(f'cov_full:\n{cov_full.numpy()}')

All eigenvalues are postive: True
mean_full:
[[0.        0.        3.3896322]]
cov_full:
[[ 1.          0.         -0.23119213]
 [ 0.          1.         -0.00218966]
 [-0.23119213 -0.00218966  0.15528679]]


In [ ]:
num_samples = 10**3

states = []
actions = []

for i in trange(num_samples):
    
    s = tf.random.normal(mean=0., stddev=1., shape=(2,))
    
    u = rbf_policy(s)
    
    states.append(s)
    actions.append(u)
    
s = tf.convert_to_tensor(states)
u = tf.convert_to_tensor(actions)

In [ ]:
su_samples = tf.concat([s, u[..., None]], axis=-1)

print('MC mean_full:')
mean_full = tf.reduce_mean(su_samples, axis=0)[None, ...]
print(mean_full.numpy())

print('MC cov_full:')
cov_full = (tf.einsum('ij, ik -> jk', su_samples, su_samples) / su_samples.shape[0])
cov_full = cov_full - (tf.einsum('ij, ik -> jk', mean_full, mean_full) / mean_full.shape[0])
print(cov_full.numpy())

## Sine Bounded RBF Policy

In [2]:
rbf_policy = RBFPolicy(2, 1, 5, dtype=tf.float32)

sine_transform = SineTransform(lower=-2, upper=10)

sb_rbf_policy = TransformedPolicy(rbf_policy, sine_transform)
sb_rbf_policy.reset()

In [3]:
loc = tf.zeros(2, dtype=tf.float32)
cov = tf.eye(2, dtype=tf.float32)

# mean_full_ = tf.convert_to_tensor([[ 0.,        0.,         -0.25994033]], dtype=tf.float32)
# cov_full_ = tf.convert_to_tensor([[1.,         0.,         0.09250697],
#  [0.,         1.,         0.06342697],
#  [0.09250697, 0.06342697, 0.16243385]], dtype=tf.float32)

# joint_dist_ = tfd.MultivariateNormalTriL(loc=mean_full_,
#                                         scale_tril=tf.linalg.cholesky(cov_full_))

mean_full, cov_full = sb_rbf_policy.match_moments(loc, cov)

print('All eigenvalues are postive:', bool(tf.reduce_all(tf.cast(tf.linalg.eig(cov_full)[0], dtype=tf.float32) > 0)))

print(f'mean_full:\n{mean_full.numpy()}')
print(f'cov_full:\n{cov_full.numpy()}')

All eigenvalues are postive: True
mean_full:
[[0.         0.         8.99164918]]
cov_full:
[[ 1.          0.         -0.04156292]
 [ 0.          1.         -0.16028283]
 [-0.04156292 -0.16028283  1.6227232 ]]


In [4]:
num_samples = 10**3

states = []
actions = []

for i in trange(num_samples):
    
#     samp = joint_dist_.sample()
#     s = samp[0, :2]
    s = tf.random.normal(mean=0., stddev=1., shape=(2,))
    
    u = sb_rbf_policy(s)
    
    states.append(s)
    actions.append(u)
    
s = tf.cast(tf.convert_to_tensor(states), tf.float64)
u = tf.convert_to_tensor(actions)

100%|██████████| 1000/1000 [00:02<00:00, 456.90it/s]


In [7]:
su_samples = tf.concat([s, u], axis=-1)

print('MC mean_full:')
mean_full = tf.reduce_mean(su_samples, axis=0)[None, ...]
print(mean_full.numpy())

print('MC cov_full:')
cov_full = (tf.einsum('ij, ik -> jk', su_samples, su_samples) / su_samples.shape[0])
cov_full = cov_full - (tf.einsum('ij, ik -> jk', mean_full, mean_full) / mean_full.shape[0])
print(cov_full)

MC mean_full:
[[0.01733977 0.00899722 8.98237663]]
MC cov_full:
tf.Tensor(
[[ 0.9984372  -0.02884954 -0.17893441]
 [-0.02884954  0.95945043 -0.22250973]
 [-0.17893441 -0.22250973  1.77884097]], shape=(3, 3), dtype=float64)


# Agent: match moments (closed form and MC)

## Add dummy data to agent

In [8]:
tf.random.set_seed(24)

rbf_policy = RBFPolicy(state_dim=2,
                       action_dim=1,
                       num_rbf_features=5,
                       dtype=tf.float64)

sine_transform = SineTransform(lower=-20,
                               upper=15)

sb_rbf_policy = TransformedPolicy(rbf_policy,
                                  sine_transform)

# rbf_policy.reset()
sb_rbf_policy.reset()

cost_transform = CosineTransform(lower=-1,
                                 upper=1)

eq_cost = EQCost(target_loc=tf.ones((1, 3)),
                 target_scale=1.,
                 transform=cost_transform,
                 dtype=tf.float64)

eq_agent = EQGPAgent(state_dim=2,
                     action_dim=1,
                     policy=sb_rbf_policy,
                     cost=eq_cost,
                     dtype=tf.float64)

# Create pendulum environment from Gym
env = Environment(name='Pendulum-v0')
env.reset()

num_episodes = 50
num_steps = 1

for episode in range(num_episodes):
    
    state = env.reset()
    
    state = np.array([np.pi, 8]) * (2 * np.random.uniform(size=(2,)) - 1)
    env.env.env.state = state
    
    
    for step in range(num_steps):
        
        action = tf.random.uniform(shape=()) * 4. - 2
        state, action, next_state = env.step(action[None].numpy())
        
        eq_agent.observe(state, action, next_state)

/Users/gergelyflamich/Documents/sbrml/pilco/pilco-venv/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


## Match moments analytically

In [9]:
state_loc = 1. * tf.ones(2, dtype=tf.float64)
state_cov = 10. * tf.eye(2, dtype=tf.float64)

# Match moments for the joint state-action distribution
mean_full, cov_full = sb_rbf_policy.match_moments(state_loc, state_cov)

# mean_full = 0. * tf.ones((1, 3), dtype=tf.float64)
# cov_full = 1. * tf.eye(3, dtype=tf.float64)

joint_dist = tfd.MultivariateNormalTriL(loc=mean_full,
                                        scale_tril=tf.linalg.cholesky(cov_full))

## Match moments by MC

In [10]:
num_samples = 10**3

means = []
covs = []
state_actions = []

# MC approx
for i in trange(num_samples):
    
    state_action = joint_dist.sample()
    
    #Note: mean is the expectation of the deltas!
    mean, cov = eq_agent.gp_posterior_predictive(state_action)
    means.append(mean)
    
    covs.append(cov)
    state_actions.append(state_action)
    
means = tf.concat(means, axis=0)
covs = tf.stack(covs, axis=0)
state_actions = tf.stack(state_actions, axis=0)

100%|██████████| 1000/1000 [00:15<00:00, 63.75it/s]


In [11]:
emp_mean = tf.reduce_mean(means, axis=0)

cov_mean_delta = tf.reduce_mean(means[:, None, :] * means[:, :, None], axis=0)
cov_mean_delta = cov_mean_delta - emp_mean * tf.transpose(emp_mean)
print(f'Cov[E(Δ | x)]:\n{cov_mean_delta}')
mean_cov_delta = tf.linalg.diag(tf.reduce_mean(covs, axis=[0, 1]))
print(f'E[Cov(Δ | x)]:\n{mean_cov_delta}')

states = state_actions[:, :, :eq_agent.state_dim]
emp_cross_cov = tf.reduce_mean(states * means[:, :, None], axis=0)
emp_cross_cov = emp_cross_cov - tf.reduce_mean(states, axis=0) * tf.reduce_mean(means[:, :, None], axis=0)
print(f"Cov[x, Δ]:\n{tf.transpose(emp_cross_cov)}")

emp_mean = tf.reduce_mean(means, axis=0) + mean_full[:, :eq_agent.state_dim]
emp_cov = cov_full[:eq_agent.state_dim, :eq_agent.state_dim] 
emp_cov = emp_cov + cov_mean_delta + mean_cov_delta + emp_cross_cov + tf.transpose(emp_cross_cov)
print(f"Emp mean:\n{emp_mean}")
print(f"Emp cov:\n{emp_cov}")

Cov[E(Δ | x)]:
[[ 4.65782609e-05 -1.53337716e-04]
 [-1.50575380e-04  5.40089927e-04]]
E[Cov(Δ | x)]:
[[0.99916862 0.        ]
 [0.         0.99916862]]
Cov[x, Δ]:
[[-0.001178    0.00455786]
 [ 0.00233944 -0.00669134]]
Emp mean:
[[1.00049665 0.99826535]]
Emp cov:
[[1.09968592e+01 6.74396022e-03]
 [6.74672256e-03 1.09863260e+01]]


In [12]:
m, c = eq_agent.match_moments(mean_full, cov_full)
print(50 * '=')
print(f"Analytic mean:\n{m}")
print(f"Analytic cov:\n{c}")

Analytic mean:
[1.00072588 0.99781502]
Analytic cov:
[[1.09955644e+01 9.66726715e-03]
 [9.66726715e-03 1.09841003e+01]]


In [13]:
num_samples = 10**3

emp_costs = []

for s in trange(num_samples):
    
    sample = joint_dist.sample()
    
    c = eq_cost(sample)
    
    emp_costs.append(c)
    
emp_costs = tf.stack(emp_costs)

100%|██████████| 1000/1000 [00:12<00:00, 80.01it/s]


In [14]:
emp_cost = tf.reduce_mean(emp_costs)
emp_cost

<tf.Tensor: shape=(), dtype=float64, numpy=0.9999656432748208>

In [15]:
eq_cost.expected_cost(loc=mean_full,
                      cov=cov_full)

<tf.Tensor: shape=(), dtype=float64, numpy=0.999949897141532>

# Checking accuracy of GP dynamics model

In [3]:
def sample_transitions_uniformly(num_episodes, num_steps, seed):
    
    np.random.seed(seed)
    
    # Create pendulum environment from Gym
    env = Environment(name='Pendulum-v0')
    env.reset()
    
    state_actions = []
    next_states = []

    for episode in range(num_episodes):

        state = env.reset()

        state = np.array([np.pi, 8]) * (2 * np.random.uniform(size=(2,)) - 1)
        env.env.env.state = state


        for step in range(num_steps):

            action = tf.random.uniform(shape=()) * 4. - 2
            state, action, next_state = env.step(action[None].numpy())
            
            state_action = np.concatenate([state, action], axis=0)
            
            state_actions.append(state_action)
            next_states.append(next_state)
            
    state_actions = np.stack(state_actions, axis=0)
    next_states = np.stack(next_states, axis=0)
            
    return state_actions, next_states

In [4]:
def evaluate_agent_dynamics(agent, test_data):
    
    test_inputs, test_outputs = test_data
    
    pred_means, pred_vars = agent.gp_posterior_predictive(test_inputs)
    pred_means = pred_means + test_inputs[:, :2]
    
    sq_diff = tf.math.squared_difference(pred_means,
                                         test_outputs)
    
    max_diff = tf.reduce_max(sq_diff ** 0.5, axis=0)
    min_diff = tf.reduce_min(sq_diff ** 0.5, axis=0)
    
    rmse = tf.reduce_mean(sq_diff, axis=0) ** 0.5
    smse = tf.reduce_mean(sq_diff / pred_vars, axis=0)
    
    rmse = [round(num, 3) for num in rmse.numpy()]
    smse = [round(num, 3) for num in smse.numpy()]
    max_diff = [round(num, 3) for num in max_diff.numpy()]
    min_diff = [round(num, 3) for num in min_diff.numpy()]
    
    print(f'RMSE: {rmse} SMSE {smse} Min {min_diff} Max {max_diff}')

In [9]:
rbf_policy = RBFPolicy(state_dim=2,
                       action_dim=1,
                       num_rbf_features=5,
                       dtype=tf.float64)
rbf_policy.reset()

cost_tr = CosineTransform(lower=-1,
                         upper=1)

eq_cost = EQCost(target_loc=tf.ones((1, 3)),
                 target_scale=1.,
                 transform=cost_tr,
                 dtype=tf.float64)

eq_agent = EQGPAgent(state_dim=2,
                     action_dim=1,
                     policy=rbf_policy,
                     cost=eq_cost,
                     dtype=tf.float64)

train_state_actions, train_next_states = sample_transitions_uniformly(100, 1, seed=0)

eq_agent.observe(train_state_actions[:, :2], train_state_actions[:, 2:3], train_next_states)

eq_agent.set_eq_scales_from_data()

/Users/gergelyflamich/Documents/sbrml/pilco/pilco-venv/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [61]:
test_data = sample_transitions_uniformly(1000, 1, seed=1)

evaluate_agent_dynamics(eq_agent, test_data)

RMSE: [0.009, 0.077] SMSE [0.004, 0.576] Min [0.0, 0.0] Max [0.068, 0.555]


In [11]:
eq_cost.expected_cost(tf.constant([[0, 1, 1]]), 1e-4 * tf.eye(3))

mean a  tf.Tensor([[0.]], shape=(1, 1), dtype=float64)
mean a plus pi/2 tf.Tensor([[1.57079637]], shape=(1, 1), dtype=float64)
loc, cov tf.Tensor([[1.      1.      0.99995]], shape=(1, 3), dtype=float64) tf.Tensor(
[[4.99949981e-09 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 9.99999975e-05 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 9.99999975e-05]], shape=(3, 3), dtype=float64)


<tf.Tensor: shape=(), dtype=float64, numpy=9.999374766178626e-05>